假设你有一个损失函数J需要最小化，你会怎么办？
    在这个例子中，我们使用一个简单的例子，即\\(J(w)=w^2-10w+25\\),这就是损失函数。其实这个函数就是\\(J=(w-5)^2\\),但是假设我们不知道这点，你只有上面第一个函数，我们来看一下怎么用TensorFlow将其最小化。类似的，我们就可以用TensorFlow自动找到损失函数最小的w和b的值。

In [1]:
import numpy as np
import tensorflow as tf

In [4]:
w = tf.Variable(0, dtype=tf.float32) #tensorflow中，我们使用Variable来设置变量
cost = tf.add(tf.add(w**2, tf.multiply(-10., w)),25) # 定义损失函数
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)#定义学习算法

init = tf.global_variables_initializer()
session = tf.Session()
session.run(init)
print(session.run(w))

0.0


In [5]:
session.run(train) # 运行一次梯度下降法
print(session.run(w))

0.1


In [6]:
# 运行1000次学习算法
for i in range(1000):
    session.run(train)
print(session.run(w))

4.99999


所以，在运行1000次优化算法后，我们发现，最终的到的w值已经非常接近我们的最优值了，我们其实知道最优值是5.0。这里，w是我们想要优化的参数，因此把它称之为变量，注意，我们需要做的就是定义一个损失函数，使用这些TensorFlow提供的基础运算的操作函数。TensorFlow知道如何对add和mutiply还有其他函数求导，这就是为什么你只需基本实现向前传播，他就能弄明白如何做反向传播和梯度计算。实际上，Tensorflow也重载了操作符，上面损失函数的定义也可以写作：
```python
cost = w**2 - 10*w + 25
```
结果是没有什么区别的。

如果，你想要最小化的函数是训练集函数又如何呢？不管你有什么训练数据x，当你训练神经网络的时候，训练数据x都会改变，如何把训练数据加入TensorFlow程序呢？
这里，我们应该这么定义(我们重新定义一下变量）：

In [11]:
w = tf.Variable(0, dtype=tf.float32) #tensorflow中，我们使用Variable来设置变量
x = tf.placeholder(dtype=tf.float32, shape=[3,1])
cost = x[0][0]*w**2 + x[1][0]*w + x[2][0]

现在，x变成了控制损失函数系数的数据。这个**placeholder**函数告诉TensorFlow我们稍后会给x提供数值。我们现在定义一组数据。

In [12]:
coefficient = np.array([[1.], [-10.], [25.]])

最后，我们需要用某种方式把这个系数数组接入到变量x中

In [13]:
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)#定义学习算法
init = tf.global_variables_initializer()
session = tf.Session()
session.run(init)

In [15]:
session.run(train, feed_dict={x:coefficient})
print(session.run(w))

0.198


In [16]:
for i in range(1000):
    session.run(train, feed_dict={x:coefficient})
print(session.run(w))

4.99999


现在，如果我们想要更换函数的系数的话就方便的多了。我们只需要改变向量x即可。

In [19]:
coefficient = np.array([[1.], [-20.], [100.]])

In [20]:
for i in range(1000):
    session.run(train, feed_dict={x:coefficient})
print(session.run(w))

10.0


在做编程练习时见到更多的是，Tensorflow中的placeholder是一个我们之后会赋值的变量，这种方式便于把训练数据加入损失方程，把数据加入损失方程用的就是这个句法。
如果你在做mini-batch梯度下降，在每次迭代时需要插入不同的mini-batch，那么每次迭代，我们就用feed_dict来喂入训练集不同的子集。 

In [21]:
session.close()